In [1]:
import pandas as pd
from subgrounds import Subgrounds
from subgrounds.subgraph import SyntheticField, FieldPath
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
from subgrounds.dash_wrappers import Graph, AutoUpdate
from subgrounds.plotly_wrappers import Figure, Bar, Indicator

In [2]:
def get_data(api_url):
    sg = Subgrounds()
    query = sg.load_subgraph(api_url)
    #Query Daily Finacials
    DailyFinancials = query.Query.financialsDailySnapshots(
        first = 150,
        orderBy = 'timestamp',
        orderDirection = 'desc'
    )
    # Datetime SyntheticField
    query.FinancialsDailySnapshot.datetime = SyntheticField(
        lambda timestamp: str(datetime.fromtimestamp(timestamp)),
        type_ = SyntheticField.STRING,
        deps = query.FinancialsDailySnapshot.timestamp
    )
    
    # Get the dataframe
    data = sg.query_df([
        DailyFinancials.datetime,
        DailyFinancials.totalValueLockedUSD,
        DailyFinancials.dailyDepositUSD,
        DailyFinancials.dailyWithdrawUSD,
        DailyFinancials.dailyBorrowUSD,
        DailyFinancials.dailyRepayUSD
    ])
    
    # return the dataframe
    return data

In [ ]:
# Build the dashboard
app = dash.Dash(__name__, external_stylesheets = [dbc.themes.SLATE])
# Declare server for Heroku deployment. Needed for Procfile.
server = app.server
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            dbc.Label('AAVE V3 MULTICHAIN DASHBOARD')
        ]),
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(id = 'select_chain',
                        multi = False,
                        clearable = True,
                        disabled = False,
                        style = {'display': True},
                        value = 'Polygon',
                        options = [
                            {'label' : 'Polygon', 'value' : 'https://api.thegraph.com/subgraphs/name/messari/aave-v3-polygon'},
                            {'label' : 'Optimism', 'value' : 'https://api.thegraph.com/subgraphs/name/messari/aave-v3-optimism'},
                            {'label' : 'Arbitrum', 'value' : 'https://api.thegraph.com/subgraphs/name/messari/aave-v3-arbitrum'},
                            {'label' : 'Avalanche', 'value' : 'https://api.thegraph.com/subgraphs/name/messari/aave-v3-avalanche'},
                            {'label' : 'Fantom', 'value' : 'https://api.thegraph.com/subgraphs/name/messari/aave-v3-fantom'},
                            {'label' : 'Harmony', 'value' : 'https://api.thegraph.com/subgraphs/name/messari/aave-v3-harmony'}
                        ])
        ])
    ], style = {'padding' : '10px'}),
    dbc.Row([
        dbc.Col([
            dcc.Graph(id = 'tvl',
                     config = {'displayModeBar' : 'hover'}),
        ]),
    ], style = {'padding' : '10px'}),
    dbc.Row([
        dbc.Col([
            dcc.Graph(
                id = 'deposits',
                config = {'displayModeBar' : 'hover'}
            )
        ], width = 6, xs = 12, sm = 12, md = 12, lg = 6, xl = 6),
        dbc.Col([
            dcc.Graph(
                id = 'withdrawals',
                config = {'displayModeBar' : 'hover'}
            )
        ], width = 6, xs = 12, sm = 12, md = 12, lg = 6, xl = 6)
    ], style = {'padding' : '10px'}),
    dbc.Row([
        dbc.Col([
            dcc.Graph(
                id = 'borrows',
                config = {'displayModeBar' : 'hover'}
            )
        ], width = 6, xs = 12, sm = 12, md = 12, lg = 6, xl = 6),
        dbc.Col([
            dcc.Graph(
                id = 'repays',
                config = {'displayModeBar' : 'hover'}
            )
        ], width = 6, xs = 12, sm = 12, md = 12, lg = 6, xl = 6)
    ], style = {'padding' : '10px'})
], style = {'backgroundColor' : '#17202a'}, fluid = True)

# callback functions
@app.callback(
    [
        Output('tvl', 'figure'),
        Output('deposits', 'figure'),
        Output('withdrawals', 'figure'),
        Output('borrows', 'figure'),
        Output('repays', 'figure')
    ],
    [
        Input('select_chain', 'value')
    ]
)

def build_the_graph(api_url):
    data = get_data(api_url)
    
    # Total Value Locked
    TVL = px.area(x = data['financialsDailySnapshots_datetime'],
                  y = data['financialsDailySnapshots_totalValueLockedUSD'],
                 )
    TVL.update_layout(
        plot_bgcolor = '#010915',
        paper_bgcolor = '#010915',
        title = {
                'text': 'TOTAL VALUE LOCKED',
                'y': 0.99,
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
        titlefont = {
                        'color': 'white',
                        'size': 15},
            xaxis = dict(title = '<b> Date </b>',
                        color='white',
                        showline=True,
                        showgrid=False,
                        showticklabels=True,
                        linecolor='white',
                        linewidth=2,
                        ticks='outside',
                        tickfont=dict(
                            family='Arial',
                            size=12,
                            color='white' 
                        )
            ),
            yaxis=dict(title='<b> Total Value Locked </b>',
                        color='white',
                        showline=True,
                        showgrid=False,
                        showticklabels=True,
                        linecolor='white',
                        linewidth=2,
                        ticks='outside',
                        tickfont=dict(
                           family='Arial',
                           size=12,
                           color='white'
                        )

            ) 
    )
    # Daily Deposits
    daily_deposits = go.Figure()
    daily_deposits.add_trace(
        go.Bar(
            x = data['financialsDailySnapshots_datetime'],
            y = data['financialsDailySnapshots_dailyDepositUSD'],
            name = "Daily Deposits",
            marker = dict(color = 'orange')
            
        )
    )
    daily_deposits.update_layout(
        plot_bgcolor = '#010915',
        paper_bgcolor = '#010915',
        title = {
                'text': 'DAILY DEPOSITS',
                'y': 0.99,
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
        titlefont = {
                        'color': 'white',
                        'size': 15},
            xaxis = dict(title = '<b> Date </b>',
                        color='white',
                        showline=True,
                        showgrid=False,
                        showticklabels=True,
                        linecolor='white',
                        linewidth=2,
                        ticks='outside',
                        tickfont=dict(
                            family='Arial',
                            size=12,
                            color='white' 
                        )
            ),
            yaxis=dict(title='<b> Daily Deposits </b>',
                        color='white',
                        showline=True,
                        showgrid=False,
                        showticklabels=True,
                        linecolor='white',
                        linewidth=2,
                        ticks='outside',
                        tickfont=dict(
                           family='Arial',
                           size=12,
                           color='white'
                        )

            )
    )
    # Daily withdrawals
    daily_withdrawals = go.Figure()
    daily_withdrawals.add_trace(
        go.Bar(
            x = data['financialsDailySnapshots_datetime'],
            y = data['financialsDailySnapshots_dailyWithdrawUSD'],
            name = "Daily Withdrawals",
            marker = dict(color = 'red')
            
        )
    )
    daily_withdrawals.update_layout(
        plot_bgcolor = '#010915',
        paper_bgcolor = '#010915',
        title = {
                'text': 'DAILY WITHDRAWALS',
                'y': 0.99,
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
        titlefont = {
                        'color': 'white',
                        'size': 15},
            xaxis = dict(title = '<b> Date </b>',
                        color='white',
                        showline=True,
                        showgrid=False,
                        showticklabels=True,
                        linecolor='white',
                        linewidth=2,
                        ticks='outside',
                        tickfont=dict(
                            family='Arial',
                            size=12,
                            color='white' 
                        )
            ),
            yaxis=dict(title='<b> Daily Withdrawals </b>',
                        color='white',
                        showline=True,
                        showgrid=False,
                        showticklabels=True,
                        linecolor='white',
                        linewidth=2,
                        ticks='outside',
                        tickfont=dict(
                           family='Arial',
                           size=12,
                           color='white'
                        )

            )
    )
    # Daily borrows
    daily_borrows = go.Figure()
    daily_borrows.add_trace(
        go.Bar(
            x = data['financialsDailySnapshots_datetime'],
            y = data['financialsDailySnapshots_dailyBorrowUSD'],
            name = "Daily Borrows",
            marker = dict(color = '#04B77A')
            
        )
    )
    daily_borrows.update_layout(
        plot_bgcolor = '#010915',
        paper_bgcolor = '#010915',
        title = {
                'text': 'DAILY BORROWINGS',
                'y': 0.99,
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
        titlefont = {
                        'color': 'white',
                        'size': 15},
            xaxis = dict(title = '<b> Date </b>',
                        color='white',
                        showline=True,
                        showgrid=False,
                        showticklabels=True,
                        linecolor='white',
                        linewidth=2,
                        ticks='outside',
                        tickfont=dict(
                            family='Arial',
                            size=12,
                            color='white' 
                        )
            ),
            yaxis=dict(title='<b> Daily Borrows </b>',
                        color='white',
                        showline=True,
                        showgrid=False,
                        showticklabels=True,
                        linecolor='white',
                        linewidth=2,
                        ticks='outside',
                        tickfont=dict(
                           family='Arial',
                           size=12,
                           color='white'
                        )

            )
    )
    # Daily repayments
    daily_repays = go.Figure()
    daily_repays.add_trace(
        go.Bar(
            x = data['financialsDailySnapshots_datetime'],
            y = data['financialsDailySnapshots_dailyRepayUSD'],
            name = "Daily Repayments",
            marker = dict(color = 'green')
            
        )
    )
    daily_repays.update_layout(
        plot_bgcolor = '#010915',
        paper_bgcolor = '#010915',
        title = {
                'text': 'DAILY REPAYEMENTS',
                'y': 0.99,
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
        titlefont = {
                        'color': 'white',
                        'size': 15},
            xaxis = dict(title = '<b> Date </b>',
                        color='white',
                        showline=True,
                        showgrid=False,
                        showticklabels=True,
                        linecolor='white',
                        linewidth=2,
                        ticks='outside',
                        tickfont=dict(
                            family='Arial',
                            size=12,
                            color='white' 
                        )
            ),
            yaxis=dict(title='<b> Daily Repayments </b>',
                        color='white',
                        showline=True,
                        showgrid=False,
                        showticklabels=True,
                        linecolor='white',
                        linewidth=2,
                        ticks='outside',
                        tickfont=dict(
                           family='Arial',
                           size=12,
                           color='white'
                        )

            )
    )
    return TVL, daily_deposits, daily_withdrawals, daily_borrows, daily_repays

app.run_server(debug = False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


C:\Users\robust computers\AppData\Local\Programs\Python\Python310\lib\site-packages\werkzeug\serving.py:700: ResourceWarning: unclosed <socket.socket fd=1372, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0>
  self.socket = socket.fromfd(fd, address_family, socket.SOCK_STREAM)
 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)
127.0.0.1 - - [06/Oct/2022 11:20:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2022 11:20:04] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_6_1m1663782170.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2022 11:20:04] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_6_1m1663782169.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2022 11:20:04] "GET /_dash-component-suites/dash/deps/react@16.v2_6_1m1663782170.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2022 11:20:04] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_6_1m1663782170.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2022 11:20:04] "GET /_d

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\robust computers\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\robust computers\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\robust computers\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\robust computers\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\robust computers\AppData\Local\Programs\Python\Python310\lib\site-packages\dash\dash.py", line 1259, in dispatch
    ctx.run(
  File "C:\Users\robust com

127.0.0.1 - - [06/Oct/2022 11:20:06] "POST /_dash-update-component HTTP/1.1" 500 -
sys:1: ResourceWarning:

unclosed socket <zmq.Socket(zmq.PUSH) at 0x27fa6acab60>

sys:1: ResourceWarning:

unclosed socket <zmq.Socket(zmq.PUSH) at 0x27fa6acada0>

sys:1: ResourceWarning:

unclosed socket <zmq.Socket(zmq.PUSH) at 0x27fa6acaaa0>

127.0.0.1 - - [06/Oct/2022 11:20:57] "POST /_dash-update-component HTTP/1.1" 200 -
sys:1: ResourceWarning:

unclosed socket <zmq.Socket(zmq.PUSH) at 0x27fa6acace0>

127.0.0.1 - - [06/Oct/2022 11:22:51] "POST /_dash-update-component HTTP/1.1" 200 -
sys:1: ResourceWarning:

unclosed socket <zmq.Socket(zmq.PUSH) at 0x27fa9a4f5e0>

127.0.0.1 - - [06/Oct/2022 11:24:07] "POST /_dash-update-component HTTP/1.1" 200 -
sys:1: ResourceWarning:

unclosed socket <zmq.Socket(zmq.PUSH) at 0x27fa98cb5e0>

127.0.0.1 - - [06/Oct/2022 11:25:47] "POST /_dash-update-component HTTP/1.1" 200 -
sys:1: ResourceWarning:

unclosed socket <zmq.Socket(zmq.PUSH) at 0x27fa98cb340>

127.0.0.1 -